In [ ]:
pip install tweepy praw python-dotenv vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.4 MB/s eta 0:00:00


In [ ]:
os.environ["REDDIT_CLIENT_ID"] = "w8vAFuTjHB6TWiaB-5muLw"
os.environ["REDDIT_CLIENT_SECRET"] = "Wlalndnv58IvLGef453qYI8JUVT0iw"
os.environ["REDDIT_USER_AGENT"]='sentiment-demo/0.1 by ivan'

In [ ]:
reddit = praw.Reddit(
    client_id=os.environ["REDDIT_CLIENT_ID"],
    client_secret=os.environ["REDDIT_CLIENT_SECRET"],
    user_agent=os.environ["REDDIT_USER_AGENT"],
)
reddit.read_only = True

# 3) Sanity check: a very simple call
try:
    print("Read-only:", reddit.read_only)
    it = reddit.subreddit("python").hot(limit=1)
    print(next(it).title)
    print("OK ✅")
except Exception as e:
    print("Failed ❌ ->", type(e).__name__, e)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Read-only: True
Sunday Daily Thread: What's everyone working on this week?
OK ✅


In [ ]:
sub = reddit.subreddit("stocks")   # choose any subreddit

# Top N new posts matching a keyword
results = []
for post in sub.search(query="nvidia", sort="new", limit=25):
    results.append({
        "id": post.id,
        "title": post.title,
        "score": post.score,
        "created_utc": post.created_utc,
        "url": post.url,
        "num_comments": post.num_comments,
        "selftext": post.selftext[:500],  # preview
    })

print(f"Fetched {len(results)} posts")
print(results[:2])

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Fetched 25 posts
[{'id': '1mzwfjs', 'title': 'NVDA Up 35% Into Earnings', 'score': 205, 'created_utc': 1756141896.0, 'url': 'https://www.reddit.com/r/stocks/comments/1mzwfjs/nvda_up_35_into_earnings/', 'num_comments': 106, 'selftext': 'Nvidia reports earnings on Wednesday (Aug 27) after the close, and the setup looks like a textbook high-stakes event. The stock is currently trading at $181.30, up over 35% in the past three months, and the technicals tell a mixed story heading into earnings. On the positive side, the long-term trend is still strong. Nvidia is trading above its 10-day ($179), 50-day ($167), and 200-day ($137) moving averages, which confirms that the uptrend remains intact and the bulls are in control on a broade'}, {'id': '1mztcbq', 'title': 'Nvidia’s new ‘robot brain’ goes on sale for $3,499 as company targets robotics for growth', 'score': 157, 'created_utc': 1756135131.0, 'url': 'https://www.reddit.com/r/stocks/comments/1mztcbq/nvidias_new_robot_brain_goes_on_sale_for

In [ ]:
if results:
    post = reddit.submission(id=results[0]["id"])
    post.comments.replace_more(limit=0)  # flatten "MoreComments"
    comments = [c.body for c in post.comments.list()[:50]]  # first 50
    print(f"Collected {len(comments)} comments")
    print(comments[:3])

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Collected 50 comments
['I held NVDA for the past 5 years, and it was always volatile in the short-term.', 'I am expecting a pull back because it always pulls back after an earnings for whatever reason but I’m holding till 2030. \n\n200 shares myself so I’m small time 😅', 'It’s either gonna rip and boost the whole market or it’s April level crash. No in-between. Hedge accordingly.']


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def score(text):
    s = analyzer.polarity_scores(text)
    return s["compound"]  # -1 (neg) .. +1 (pos)

# Example: score tweets
if 'all_rows' in globals() and all_rows:
    for r in all_rows[:10]:
        r["sentiment"] = score(r["text"])
    print(all_rows[:3])

# Example: score reddit comments
if 'comments' in globals() and comments:
    scored = [{"text": c[:120]+"...", "sentiment": score(c)} for c in comments[:10]]
    print(scored[:3])

[{'text': 'I held NVDA for the past 5 years, and it was always volatile in the short-term....', 'sentiment': 0.0}, {'text': 'I am expecting a pull back because it always pulls back after an earnings for whatever reason but I’m holding till 2030....', 'sentiment': 0.7227}, {'text': 'It’s either gonna rip and boost the whole market or it’s April level crash. No in-between. Hedge accordingly....', 'sentiment': -0.296}]
